## 訓練 CBoW 模型

此筆記本是 [AI for Beginners Curriculum](http://aka.ms/ai-beginners) 的一部分

在這個範例中，我們將學習如何訓練 CBoW 語言模型，以建立我們自己的 Word2Vec 嵌入空間。我們將使用 AG News 數據集作為文本來源。


In [30]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

我們將開始載入數據集：


In [1]:
ds_train, ds_test = tfds.load('ag_news_subset').values()

## CBoW 模型

CBoW 模型學習根據 $2N$ 個鄰近的詞來預測一個詞。例如，當 $N=1$ 時，我們可以從句子 *I like to train networks* 中得到以下配對：(like, I)、(I, like)、(to, like)、(like, to)、(train, to)、(to, train)、(networks, train)、(train, networks)。在這裡，第一個詞是作為輸入的鄰近詞，第二個詞是我們要預測的詞。

為了構建一個用於預測下一個詞的網絡，我們需要提供鄰近詞作為輸入，並輸出詞的編號。CBoW 網絡的架構如下：

* 輸入的詞會通過嵌入層（embedding layer）。這個嵌入層就是我們的 Word2Vec 嵌入，因此我們會將其單獨定義為 `embedder` 變數。在這個例子中，我們將使用嵌入維度大小為 30，儘管你可能想嘗試更高的維度（真實的 Word2Vec 使用的是 300 維）。
* 嵌入向量接著會傳遞到一個全連接層（dense layer），該層負責預測輸出詞。因此它包含 `vocab_size` 個神經元。

Keras 中的嵌入層會自動知道如何將數字輸入轉換為 one-hot 編碼，因此我們不需要單獨對輸入詞進行 one-hot 編碼。我們指定 `input_length=1`，表示我們的輸入序列中只需要一個詞——通常嵌入層是為處理更長的序列而設計的。

對於輸出，如果我們使用 `sparse_categorical_crossentropy` 作為損失函數，那麼我們只需要提供詞的編號作為預期結果，而不需要進行 one-hot 編碼。

我們將 `vocab_size` 設置為 5000，以減少計算量。我們還會定義一個稍後會用到的向量化工具（vectorizer）。


In [68]:
vocab_size = 5000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))
embedder = keras.layers.Embedding(vocab_size,30,input_length=1)

model = keras.Sequential([
    embedder,
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 30)             150000    
                                                                 
 dense_1 (Dense)             (None, 1, 5000)           155000    
                                                                 
Total params: 305,000
Trainable params: 305,000
Non-trainable params: 0
_________________________________________________________________


讓我們初始化向量化器並提取詞彙表：


In [69]:
def extract_text(x):
    return x['title']+' '+x['description']

vectorizer.adapt(ds_train.take(500).map(extract_text))
vocab = vectorizer.get_vocabulary()

## 準備訓練數據

現在讓我們編寫主要函式，用於從文本中計算 CBoW 詞對。這個函式將允許我們指定窗口大小，並返回一組詞對——輸入詞和輸出詞。請注意，這個函式既可以用於詞，也可以用於向量/張量——這將使我們能夠在傳遞給 `to_cbow` 函式之前對文本進行編碼。


In [70]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(vectorizer('I like to train networks')))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=771>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=3>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=376>], [<tf.Tensor: shape=(), dtype=int64, numpy=771>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=376>, <tf.Tensor: shape=(), dtype=int64, numpy=3>], [<tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=3>]

讓我們準備訓練數據集。我們將遍歷所有新聞，調用 `to_cbow` 以獲取詞對列表，並將這些詞對添加到 `X` 和 `Y` 中。為了節省時間，我們只考慮前 10k 條新聞項目——如果你有更多時間等待並希望獲得更好的嵌入，可以輕鬆移除此限制 :)


In [100]:
X = []
Y = []
for i,x in zip(range(10000),ds_train.map(extract_text).as_numpy_iterator()):
    for w1, w2 in to_cbow(vectorizer(x),window_size=1):
        X.append(tf.expand_dims(w1,0))
        Y.append(tf.expand_dims(w2,0))

我們還會將該數據轉換為一個數據集，並將其分批進行訓練：


In [101]:
ds = tf.data.Dataset.from_tensor_slices((X,Y)).batch(256)

現在讓我們進行實際訓練。我們將使用 `SGD` 優化器，並設定相當高的學習率。你也可以嘗試使用其他優化器，例如 `Adam`。我們將先訓練 200 個世代——如果你希望獲得更低的損失，可以重新執行此單元格。


In [102]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.1),loss='sparse_categorical_crossentropy')
model.fit(ds,epochs=200)

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2156/2156 [==============================] - 7s 3ms/step - loss: 5.6134
Epoch 2/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5431
Epoch 3/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.5029
Epoch 4/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4754
Epoch 5/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4548
Epoch 6/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4382
Epoch 7/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4243
Epoch 8/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4123
Epoch 9/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.4019
Epoch 10/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3926
Epoch 11/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3843
Epoch 12/200
2156/2156 [==============================] - 7s 3ms/step - loss: 5.3767
E

## 嘗試使用 Word2Vec

要使用 Word2Vec，讓我們提取與詞彙表中所有單詞對應的向量：


In [103]:
vectors = embedder(vectorizer(vocab))
vectors = tf.reshape(vectors,(-1,30)) # we need reshape to get rid of extra dimension

讓我們看看，例如，單詞 **Paris** 是如何編碼成向量的：


In [104]:
paris_vec = embedder(vectorizer('paris'))[0]
print(paris_vec)

tf.Tensor(
[-0.13308628  0.50972325  0.00344684  0.185389   -0.03176536  0.22262476
 -0.3856765  -0.6854793   0.5185803  -0.7215402  -0.16101503  0.15622072
  0.00653811 -0.14954254  0.03379822 -0.01243829  0.27907634 -0.32538188
  0.21718933  0.31112966 -0.24142407  0.15589055  0.2915561   0.19029242
  0.08425518 -0.0941902  -0.54313695 -0.24854654  0.26196313  0.18027727], shape=(30,), dtype=float32)


使用 Word2Vec 來尋找同義詞是很有趣的。以下函數將返回與給定輸入最接近的 `n` 個單詞。為了找到它們，我們計算 $|w_i - v|$ 的範數，其中 $v$ 是對應於我們輸入單詞的向量，而 $w_i$ 是詞彙表中第 $i$ 個單詞的編碼。我們接著對數組進行排序，並使用 `argsort` 返回對應的索引，然後取列表的前 `n` 個元素，這些元素編碼了詞彙表中最接近單詞的位置。


In [105]:
def close_words(x,n=5):
  vec = embedder(vectorizer(x))[0]
  top5 = np.linalg.norm(vectors-vec,axis=1).argsort()[:n]
  return [ vocab[x] for x in top5 ]

close_words('paris')

['paris', 'philippines', 'seoul', 'jakarta', 'zoo']

In [112]:
close_words('china')

['china', 'russia', 'pakistan', 'israel', 'turkey']

In [113]:
close_words('official')

['official', 'military', 'office', 'police', 'sources']

## 重點

透過使用像 CBoW 這樣的巧妙技術，我們可以訓練 Word2Vec 模型。你也可以嘗試訓練 skip-gram 模型，該模型是基於給定中心詞來預測鄰近詞，看看它的表現如何。



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
